In [13]:
import numpy as np
from scipy.sparse import csr_matrix
from multiprocessing import Pool, cpu_count
import multiprocessing

def crear_matriz_sparse(filas, columnas, densidad):
    # Calcular el número de elementos no nulos como entero
    nnz = int(filas * columnas * densidad)

    # Generar datos aleatorios para la matriz sparse
    datos = np.random.randint(1, 10, size=nnz)
    indices_filas = np.random.randint(0, filas, size=nnz)
    indices_columnas = np.random.randint(0, columnas, size=nnz)

    # Crear y retornar la matriz sparse
    matriz = csr_matrix((datos, (indices_filas, indices_columnas)), shape=(filas, columnas))
    return matriz

def multiplicar_fila(args):
    matriz1, matriz2, fila = args
    # Multiplicar una fila de matriz1 por matriz2
    resultado_fila = matriz1.getrow(fila) * matriz2
    return resultado_fila

def multiplicacion_paralela(matriz1, matriz2, operacion='fila'):
    num_trabajos = matriz1.shape[0] if operacion == 'fila' else matriz2.shape[1]
    pool = Pool(processes=multiprocessing.cpu_count())

    if operacion == 'fila':
        funcion_multiplicar = multiplicar_fila
        resultados = pool.map(funcion_multiplicar, [(matriz1, matriz2, i) for i in range(num_trabajos)])
        pool.close()
        pool.join()
        return resultados  # Devolver una lista de matrices sparse

    elif operacion == 'columna':
        # Transponer matriz2 para multiplicación por columnas
        matriz2_transpuesta = matriz2.transpose()
        funcion_multiplicar = multiplicar_fila
        resultados = pool.map(funcion_multiplicar, [(matriz1, matriz2_transpuesta, i) for i in range(num_trabajos)])
        pool.close()
        pool.join()
        return resultados  # Devolver una lista de matrices sparse

    else:
        raise ValueError("Operación no soportada. Elija 'fila' o 'columna'.")

# Ejemplo de uso:
filas = 1000
columnas = 1000
densidad = 0.01  # Densidad de elementos no nulos (ajustable según tus necesidades)

matriz1 = crear_matriz_sparse(filas, columnas, densidad)
matriz2 = crear_matriz_sparse(columnas, filas, densidad)

# Multiplicar matrices sparse en paralelo por filas
resultado_filas_paralelo = multiplicacion_paralela(matriz1, matriz2, operacion='fila')

# Multiplicar matrices sparse en paralelo por columnas
resultado_columnas_paralelo = multiplicacion_paralela(matriz1, matriz2, operacion='columna')

# Imprimir la primera matriz sparse en formato CSR para verificar
print("Ejemplo de matriz sparse en formato CSR:")
print(resultado_filas_paralelo[0])

# Si deseas ver la representación densa, puedes convertir una matriz sparse a formato denso así:
print("\nRepresentación densa de la primera matriz sparse:")
print(resultado_filas_paralelo[0].toarray())


Ejemplo de matriz sparse en formato CSR:
  (0, 989)	15
  (0, 972)	6
  (0, 874)	21
  (0, 772)	12
  (0, 725)	24
  (0, 713)	6
  (0, 540)	15
  (0, 365)	6
  (0, 364)	18
  (0, 311)	21
  (0, 197)	21
  (0, 919)	45
  (0, 604)	9
  (0, 381)	45
  (0, 276)	18
  (0, 184)	36
  (0, 180)	36
  (0, 126)	54
  (0, 909)	1
  (0, 863)	2
  (0, 803)	9
  (0, 797)	3
  (0, 623)	3
  (0, 488)	3
  (0, 635)	9
  :	:
  (0, 800)	5
  (0, 747)	10
  (0, 705)	4
  (0, 693)	3
  (0, 426)	8
  (0, 362)	5
  (0, 330)	9
  (0, 239)	1
  (0, 228)	9
  (0, 179)	5
  (0, 176)	8
  (0, 160)	8
  (0, 140)	6
  (0, 123)	6
  (0, 120)	48
  (0, 807)	20
  (0, 796)	20
  (0, 754)	8
  (0, 724)	8
  (0, 673)	4
  (0, 667)	8
  (0, 627)	12
  (0, 265)	16
  (0, 102)	32
  (0, 35)	24

Representación densa de la primera matriz sparse:
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0 24  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0 63  0  0  0 72  0  0  0  0  0  0  0  0  0  0  0
   0  0  